In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import trglobals
from amharic_segmenter import AmharicSegmenter

In [3]:
sent_punct = []
word_punct = []
seg = AmharicSegmenter(sent_punct, word_punct)

In [4]:
seg

In [5]:
test_sent = """ምዕራፍ አሥራ አራት
የሥራ ክፍፍልና ማፈብረክ
1.
የማፈብረክ ጥንድ ምንጭ
በሥራ ክፍፍል ላይ የተመሠረተ ኅብረትና ትክክለኛ ገጽታው የሚታየው
በማፈብረክ ውስጥ ሲሆን º የማፈብረክ ወቅት ተብሎ በሚጠራው ጊዜ ውስጥ
ሁሉ የካፒታሊስት የምርት ሂደት ዓይነተኛ መልክ ሆኖ ይገኛል " በአጠቃላይ
አነጋገር ' ያ ወቅት ከ16ኛው ምዕት ዓመት አጋማሽ እስከ 18ኛው ምዕት ዓመት
መጨረሻ ሢሶ ድረስ ያለውን ያጠቃልላል ።"""

In [6]:
result = seg.tokenize_sentence(test_sent)

### Ok, Hunalign dont work for Amharic. So just... manually do it?

In [7]:
import glob

import pandas as pd

import trglobals

In [8]:
# Load another alignment and just drop the Amharic sentences in
en_fpaths = glob.glob("../Texts_Aligned/de_tge.en_fowkes/*.txt")

In [9]:
cur_fpath = en_fpaths[0]

In [10]:
cur_df = pd.read_csv(cur_fpath, delimiter="\t", header=None)

In [11]:
cur_df.columns = ['am','en','alignment_id']

### Load Amharic texts

In [12]:
am_fpaths = glob.glob("../Texts/Amharic_Capital/cleaned/*.txt")

### But... can do better.

The AmharicSegmenter can't detect OCR error punctuation, so here I manually break on that

Cool. So, much closer to the expected # of sents. Now we... force the aligner's hand by just putting the period symbol at the end of each detected sent

In [13]:
custom_output_path = "../Texts/Amharic_Capital/cleaned_customsplit/"
if not os.path.isdir(custom_output_path):
    os.mkdir(custom_output_path)

In [14]:
def load_and_tokenize(am_fpath):
    with open(am_fpath, 'r', encoding='utf-8') as infile:
        am_text = infile.read()
    am_sents, am_sents_nocustom = trglobals._am_sent_tokenize(am_text, return_nocustom=True)
    return am_sents, am_sents_nocustom

In [15]:
# And now we can make a new custom version of each chapter
all_ch_sents_am = []
for cur_am_fpath in am_fpaths:
    # Get the fname prefix
    cur_fname = os.path.basename(cur_am_fpath)
    cur_sents, cur_sents_nocustom = load_and_tokenize(cur_am_fpath)
    num_sents_tokenized = len(cur_sents)
    if num_sents_tokenized > 1500:
        print(f"Tokenization issue: num_sents_tokenized = {num_sents_tokenized}")
        print(f"Non-custom num sents: {len(cur_sents_nocustom)}")
        # So, in this case, we just use the original non-custom tokenization
        cur_sents = cur_sents_nocustom
    all_ch_sents_am.append(cur_sents)
    sent_str = "\n".join(cur_sents)
    cur_output_fpath = os.path.join(custom_output_path, cur_fname)
    # And save
    with open(cur_output_fpath, 'w', encoding='utf-8') as outfile:
        outfile.write(sent_str)
    print(f"Written to {cur_output_fpath}")

Written to ../Texts/Amharic_Capital/cleaned_customsplit/ch01_clean.am_iish.txt
Written to ../Texts/Amharic_Capital/cleaned_customsplit/ch02_clean.am_iish.txt
Written to ../Texts/Amharic_Capital/cleaned_customsplit/ch03_clean.am_iish.txt
Written to ../Texts/Amharic_Capital/cleaned_customsplit/ch04_clean.am_iish.txt
Written to ../Texts/Amharic_Capital/cleaned_customsplit/ch05_clean.am_iish.txt
Written to ../Texts/Amharic_Capital/cleaned_customsplit/ch06_clean.am_iish.txt
Written to ../Texts/Amharic_Capital/cleaned_customsplit/ch07_clean.am_iish.txt
Written to ../Texts/Amharic_Capital/cleaned_customsplit/ch08_clean.am_iish.txt
Written to ../Texts/Amharic_Capital/cleaned_customsplit/ch09_clean.am_iish.txt
Written to ../Texts/Amharic_Capital/cleaned_customsplit/ch10_clean.am_iish.txt
Written to ../Texts/Amharic_Capital/cleaned_customsplit/ch11_clean.am_iish.txt
Written to ../Texts/Amharic_Capital/cleaned_customsplit/ch12_clean.am_iish.txt
Written to ../Texts/Amharic_Capital/cleaned_customsp

### And now, as a test, just try "aligning" Amharic and German via sentences

In [16]:
trglobals.text_paths.keys()

dict_keys(['am_iish', 'en_fowkes', 'en_aveling', 'de_tge', 'ru_morg', 'zh_morg'])

In [17]:
de_fpaths = glob.glob(os.path.join(trglobals.text_paths['de_tge'], "*.txt"))
num_de_fpaths = len(de_fpaths)

In [18]:
all_ch_sents_de = []
all_fnames_de = []
for cur_de_fpath in de_fpaths:
    cur_de_fname = os.path.basename(cur_de_fpath)
    all_fnames_de.append(cur_de_fname)
    with open(cur_de_fpath, 'r', encoding='utf-8') as infile:
        cur_de_text = infile.read()
    cur_de_sents = trglobals.sent_tokenize(cur_de_text, "de")
    cur_sents_clean = [trglobals.remove_footnotes(cur_sent, "de") for cur_sent in cur_de_sents]
    all_ch_sents_de.append(cur_sents_clean)


In [19]:
all_ch_names = [fname.split(".")[0] for fname in all_fnames_de]

In [20]:
sent_ratios = {}
for cur_ch_index in range(num_de_fpaths):
    cur_ch_name = all_ch_names[cur_ch_index]
    print(f"Processing {cur_ch_name}")
    cur_de_sents = all_ch_sents_de[cur_ch_index]
    num_de_sents = len(cur_de_sents)
    print(f"de sents: {num_de_sents}")
    cur_am_sents = all_ch_sents_am[cur_ch_index]
    num_am_sents = len(cur_am_sents)
    print(f"am sents: {num_am_sents}")
    # Print warning if bad ratio...
    sent_ratio = num_am_sents / num_de_sents
    sent_ratios[cur_ch_name] = (sent_ratio, num_am_sents, num_de_sents)
    if sent_ratio > 1.5:
        print(f"*** sent imbalance: ratio = {sent_ratio} ***")
    # Trying to fix this... try original non-custom split in this case?

    longer_len = max(num_de_sents, num_am_sents)
    blank_rows = [""] * longer_len
    cur_ch_df = pd.DataFrame({'de': blank_rows, 'am': blank_rows})
    cur_ch_df.iloc[:num_de_sents, cur_ch_df.columns.get_loc('de')] = cur_de_sents
    cur_ch_df.iloc[:num_am_sents, cur_ch_df.columns.get_loc('am')] = cur_am_sents
    custom_aligned_path = "../Texts_Aligned/de_tge.am_iish2/"
    if not os.path.isdir(custom_aligned_path):
        os.mkdir(custom_aligned_path)
    # And save the "alignment"
    cur_ch_name = all_ch_names[cur_ch_index]
    cur_aligned_fpath = os.path.join(custom_aligned_path, f"{cur_ch_name}.csv")
    cur_ch_df.to_csv(cur_aligned_fpath)
    print(f"Saved {cur_aligned_fpath}")

Processing ch01
de sents: 697
am sents: 450
Saved ../Texts_Aligned/de_tge.am_iish2/ch01.csv
Processing ch02
de sents: 122
am sents: 114
Saved ../Texts_Aligned/de_tge.am_iish2/ch02.csv
Processing ch03
de sents: 685
am sents: 374
Saved ../Texts_Aligned/de_tge.am_iish2/ch03.csv
Processing ch04
de sents: 466
am sents: 93
Saved ../Texts_Aligned/de_tge.am_iish2/ch04.csv
Processing ch05
de sents: 168
am sents: 85
Saved ../Texts_Aligned/de_tge.am_iish2/ch05.csv
Processing ch06
de sents: 130
am sents: 70
Saved ../Texts_Aligned/de_tge.am_iish2/ch06.csv
Processing ch07
de sents: 361
am sents: 172
Saved ../Texts_Aligned/de_tge.am_iish2/ch07.csv
Processing ch08
de sents: 162
am sents: 79
Saved ../Texts_Aligned/de_tge.am_iish2/ch08.csv
Processing ch09
de sents: 339
am sents: 153
Saved ../Texts_Aligned/de_tge.am_iish2/ch09.csv
Processing ch10
de sents: 864
am sents: 656
Saved ../Texts_Aligned/de_tge.am_iish2/ch10.csv
Processing ch11
de sents: 106
am sents: 42
Saved ../Texts_Aligned/de_tge.am_iish2/ch

In [21]:
sent_ratios

{'ch01': (0.6456241032998565, 450, 697),
 'ch02': (0.9344262295081968, 114, 122),
 'ch03': (0.545985401459854, 374, 685),
 'ch04': (0.19957081545064378, 93, 466),
 'ch05': (0.5059523809523809, 85, 168),
 'ch06': (0.5384615384615384, 70, 130),
 'ch07': (0.47645429362880887, 172, 361),
 'ch08': (0.4876543209876543, 79, 162),
 'ch09': (0.45132743362831856, 153, 339),
 'ch10': (0.7592592592592593, 656, 864),
 'ch11': (0.39622641509433965, 42, 106),
 'ch12': (0.8125, 104, 128),
 'ch13': (0.3877551020408163, 57, 147),
 'ch14': (0.6861538461538461, 223, 325),
 'ch15': (0.7380281690140845, 1310, 1775),
 'ch16': (0.903448275862069, 131, 145),
 'ch17': (0.6384180790960452, 113, 177),
 'ch18': (0.32653061224489793, 16, 49),
 'ch19': (0.6944444444444444, 75, 108),
 'ch20': (0.5824175824175825, 53, 91),
 'ch21': (0.41935483870967744, 39, 93),
 'ch22': (0.7547169811320755, 40, 53),
 'ch23': (0.8571428571428571, 144, 168),
 'ch24': (0.6549295774647887, 279, 426),
 'ch25': (0.7302075326671791, 950, 13

In [22]:
# Sort by ratio
ratios_sorted = sorted(sent_ratios.items(), key=lambda x: x[1][0])

In [23]:
ratios_sorted

[('ch04', (0.19957081545064378, 93, 466)),
 ('ch18', (0.32653061224489793, 16, 49)),
 ('ch13', (0.3877551020408163, 57, 147)),
 ('ch11', (0.39622641509433965, 42, 106)),
 ('ch21', (0.41935483870967744, 39, 93)),
 ('ch09', (0.45132743362831856, 153, 339)),
 ('ch07', (0.47645429362880887, 172, 361)),
 ('ch08', (0.4876543209876543, 79, 162)),
 ('ch05', (0.5059523809523809, 85, 168)),
 ('ch30', (0.5283018867924528, 28, 53)),
 ('ch06', (0.5384615384615384, 70, 130)),
 ('ch03', (0.545985401459854, 374, 685)),
 ('ch20', (0.5824175824175825, 53, 91)),
 ('ch17', (0.6384180790960452, 113, 177)),
 ('ch01', (0.6456241032998565, 450, 697)),
 ('ch24', (0.6549295774647887, 279, 426)),
 ('ch28', (0.6854838709677419, 85, 124)),
 ('ch14', (0.6861538461538461, 223, 325)),
 ('ch19', (0.6944444444444444, 75, 108)),
 ('ch32', (0.7105263157894737, 27, 38)),
 ('ch25', (0.7302075326671791, 950, 1301)),
 ('ch15', (0.7380281690140845, 1310, 1775)),
 ('ch26', (0.74, 37, 50)),
 ('ch22', (0.7547169811320755, 40, 53

In [24]:
# So... for ch12 (ratio 2416:128) and ch19 (ratio 1236:108), prob best to do manually.
# Otherwise, not bad!

In [25]:
# Loading these chs and just having their am column be a single line, for splitting into chunks manually
custom_chs = ['ch12', 'ch19']

In [26]:
for cur_custom_ch in custom_chs:
    cur_fpath = f"../Texts/Amharic_Capital/cleaned/{cur_custom_ch}_clean.am_iish.txt"
    with open(cur_fpath, 'r', encoding='utf-8') as infile:
        cur_text = infile.read()
    print(f"Num periods: {cur_text.count(trglobals.AM_PERIOD)}")

Num periods: 63
Num periods: 28


In [27]:
cur_text

"ክፍል ስድስት ም ን ዳ ምዕራፍ Gስራ ዘጠኝ የሥራ ኃይል እሴት (እንዲሁም የዋጋ) ወዶ ምንዳ መለወጥ በቡርዧ ኅብረተሰብ ውስጥምንዳ ላይ ላዩን ሲታይ የሥራ ዋጋ ማለትምለተ ወሰነ የሥራ መጠን የተከፈለ የተወሰነ የገንዘብ መጠን ይመስላል = እንዲህም በመሆኑ ሰዎች የሥራኃይል እሴት አድርገውይቆጥሩታል። ገንዛባዊ መልኩንም አስፈላጊ ወይም መደበኛ ዋጋው አድርገው ይቆጥሩታል = በሌላ በኩልም የሥራን የገበያ ዋጋ ከመደበኛ እርከኑ በላይ ወይም በታች የሚዋዥቅ ዋጋ አድ ርገው ይናገራሉ ። ለመሆኑ የአንድ ሸቀጥ እሴት ምንድን ነው ? ሸቀጡን ለማምረት የዋለው የማኅበራዊ ሥራ ነባራዊ መልኩ ነው የዚህን እሴት መጠንስ እንዴት እንለካ ዋለን ? ውስጡ በአለው የሥራ መጠን ታዲያ ለምሳሌ የ12 ሰዓት የሥራ ቀን እሴት የሚወሰነው እንዴት ነው ? በ12 ሰዓት የሥራ ቀን ውስጥ ባሉት 12 የሥራ ሰዓቶች ነው = ይህ ትርጉም የለሽ ድግግሞሽ ነው ። ጉልበት ሸቀጥ ሆኖ ለሽያጭ እንዲቀርብ ከተፈለገ ከመሸጡ በፊት የግድ መኖር አለበት ዳሩ ግን ሠራተኛው ጉልበትን ራሱን የቻለ ነባራዊ ህልውና ያለው ነገር አድርጐ ለማቅረብ ቢችል ኖሮ የሚሸጠው ሸቀጥን እንጂ ጉልበትን አይሆንም ነበር ውን የሚያቃውስ መስሎ የሚታየውን ችግር ሪካርዶ በዴ ያስወደዋል 1ይተወዋል - ይህ መር “ እሴት በምርት ውስጥ በዋለ የሥራ መጠን ላይ የተመሠረተ ነው የሚለውን አስተምህሮ በትhhል ተግባራዊ ከሆነ የሥራ እሴት እሱን ራሱን ለማስገኘት በፈሰሰው የሥራ መጠን ላይ የተ መሠረተ ነው ወደሚል አስቂኝ መደምደሚያ ያመራል ስለዚህም ሪካርዶ ቅልብስ ይልና የሥራን እሴት ምንዳን ለማስገኘት በፈሰሰው የሥራ መጠን ላይ የተመሠረተ ያደርገዋል ወይም በእርሱ አባባል መሠረት የሥራ እሴት የሚወሰነው ምንዳን ለማስገኘት በፈሰሰው ጉልበት መጠን

In [28]:
import re

In [31]:
am_punct_reg = re.compile(r' " | = |።|\?| " | \* | “ ', re.UNICODE)

In [32]:
am_punct_reg.split("ክፍል ስድስት ም ን ዳ ምዕራፍ Gስራ ዘጠኝ የሥራ ኃይል እሴት (እንዲሁም የዋጋ) ወዶ ምንዳ መለወጥ በቡርዧ ኅብረተሰብ ውስጥምንዳ ላይ ላዩን ሲታይ የሥራ ዋጋ ማለትምለተ ወሰነ የሥራ መጠን የተከፈለ የተወሰነ የገንዘብ መጠን ይመስላል = እንዲህም በመሆኑ ሰዎች የሥራኃይል እሴት አድርገውይቆጥሩታል። ገንዛባዊ መልኩንም አስፈላጊ ወይም መደበኛ ዋጋው አድርገው ይቆጥሩታል = በሌላ በኩልም የሥራን የገበያ ዋጋ ከመደበኛ እርከኑ በላይ ወይም በታች የሚዋዥቅ ዋጋ አድ ርገው ይናገራሉ ። ለመሆኑ የአንድ ሸቀጥ እሴት ምንድን ነው ? ሸቀጡን ለማምረት የዋለው የማኅበራዊ ሥራ ነባራዊ መልኩ ነው የዚህን እሴት መጠንስ እንዴት እንለካ ዋለን ? ውስጡ በአለው የሥራ መጠን ታዲያ ለምሳሌ የ12 ሰዓት የሥራ ቀን እሴት የሚወሰነው እንዴት ነው ? በ12 ሰዓት የሥራ ቀን ውስጥ ባሉት 12 የሥራ ሰዓቶች ነው = ይህ ትርጉም የለሽ ድግግሞሽ ነው ። ጉልበት ሸቀጥ ሆኖ ለሽያጭ እንዲቀርብ ከተፈለገ ከመሸጡ በፊት የግድ መኖር አለበት ዳሩ ግን ሠራተኛው ጉልበትን ራሱን የቻለ ነባራዊ ህልውና ያለው ነገር አድርጐ ለማቅረብ ቢችል ኖሮ የሚሸጠው ሸቀጥን እንጂ ጉልበትን አይሆንም ነበር ውን የሚያቃውስ መስሎ የሚታየውን ችግር ሪካርዶ በዴ ያስወደዋል 1ይተወዋል - ይህ መር “ እሴት በምርት ውስጥ በዋለ የሥራ መጠን ላይ የተመሠረተ ነው የሚለውን አስተምህሮ በትhhል ተግባራዊ ከሆነ የሥራ እሴት እሱን ራሱን ለማስገኘት በፈሰሰው የሥራ መጠን ላይ የተ መሠረተ ነው ወደሚል አስቂኝ መደምደሚያ ያመራል ስለዚህም ሪካርዶ ቅልብስ ይልና የሥራን እሴት ምንዳን ለማስገኘት በፈሰሰው የሥራ መጠን ላይ የተመሠረተ ያደርገዋል ወይም በእርሱ አባባል መሠረት የሥራ እሴት የሚወሰነው ምንዳን ለማስገኘት በፈሰሰው ጉልበት መጠን ነው * ይህን ሲልም ለሠራተኛው የሚሰጠውን ገንዘብ ወይም ሸቀጥ ለማስገኘት የዋለውን የሥራ መጠን ልብሱ የሚገዛበትን ብር ከማዕድን ለማውጣት በፈሰሰው የሥራ መጠን ነው እንደማለትይቆጠራል” ማለቱ ነœ ይህም የልብስ እሴት የሚወሰነው ልብሱን ለማምረት በዋለው የሥራ መጠን ሳይሆን (A Critical Dissertation on the Nature, &c., of Value, pp. 50-51) “ ጉል")

['ክፍል ስድስት ም ን ዳ ምዕራፍ Gስራ ዘጠኝ የሥራ ኃይል እሴት (እንዲሁም የዋጋ) ወዶ ምንዳ መለወጥ በቡርዧ ኅብረተሰብ ውስጥምንዳ ላይ ላዩን ሲታይ የሥራ ዋጋ ማለትምለተ ወሰነ የሥራ መጠን የተከፈለ የተወሰነ የገንዘብ መጠን ይመስላል',
 'እንዲህም በመሆኑ ሰዎች የሥራኃይል እሴት አድርገውይቆጥሩታል',
 ' ገንዛባዊ መልኩንም አስፈላጊ ወይም መደበኛ ዋጋው አድርገው ይቆጥሩታል',
 'በሌላ በኩልም የሥራን የገበያ ዋጋ ከመደበኛ እርከኑ በላይ ወይም በታች የሚዋዥቅ ዋጋ አድ ርገው ይናገራሉ ',
 ' ለመሆኑ የአንድ ሸቀጥ እሴት ምንድን ነው ',
 ' ሸቀጡን ለማምረት የዋለው የማኅበራዊ ሥራ ነባራዊ መልኩ ነው የዚህን እሴት መጠንስ እንዴት እንለካ ዋለን ',
 ' ውስጡ በአለው የሥራ መጠን ታዲያ ለምሳሌ የ12 ሰዓት የሥራ ቀን እሴት የሚወሰነው እንዴት ነው ',
 ' በ12 ሰዓት የሥራ ቀን ውስጥ ባሉት 12 የሥራ ሰዓቶች ነው',
 'ይህ ትርጉም የለሽ ድግግሞሽ ነው ',
 ' ጉልበት ሸቀጥ ሆኖ ለሽያጭ እንዲቀርብ ከተፈለገ ከመሸጡ በፊት የግድ መኖር አለበት ዳሩ ግን ሠራተኛው ጉልበትን ራሱን የቻለ ነባራዊ ህልውና ያለው ነገር አድርጐ ለማቅረብ ቢችል ኖሮ የሚሸጠው ሸቀጥን እንጂ ጉልበትን አይሆንም ነበር ውን የሚያቃውስ መስሎ የሚታየውን ችግር ሪካርዶ በዴ ያስወደዋል 1ይተወዋል - ይህ መር',
 'እሴት በምርት ውስጥ በዋለ የሥራ መጠን ላይ የተመሠረተ ነው የሚለውን አስተምህሮ በትhhል ተግባራዊ ከሆነ የሥራ እሴት እሱን ራሱን ለማስገኘት በፈሰሰው የሥራ መጠን ላይ የተ መሠረተ ነው ወደሚል አስቂኝ መደምደሚያ ያመራል ስለዚህም ሪካርዶ ቅልብስ ይልና የሥራን እሴት ምንዳን ለማስገኘት በፈሰሰው የሥራ መጠን ላይ የተመሠረተ ያደርገዋል ወይም በእርሱ አባባል መሠረት የሥራ እሴት 